# Data Cleaning Listado de Listado de Beneficiarios 2023

## Autores
- José Luis Delgado Dávara
- Arturo Ortiz Aguilar
- Beltrán Valle Gutiérrez-Cortines

## Importante leer para entender

En este Notebook se trabaja con 3 listados importantes:

1. Beneficiarios2023 -> Dataset con el listado de TODOS los beneficiarios.
2. Estados_Beneficiarios 2023 -> Dataset sólo con los estados *únicos* encontrados en el dataset anterior.
3. Diccionario ->
4. Dataset_Inegi ->

In [1]:
import pandas as pd
import seaborn as sns
from thefuzz import fuzz
from thefuzz import process
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import re
import unidecode

In [2]:
def load_datasets(directory):
    # Get a list of all CSV files in the directory
    csv_files = glob.glob(os.path.join(directory, '*.csv'))

    # Read each CSV file and store the DataFrame in a list
    dataframes = [pd.read_csv(file, encoding='cp1252', index_col=0, skiprows=1) for file in csv_files]

    # Print the number of rows for each DataFrame
    for i, df in enumerate(dataframes):
        print(f"Number of rows in DataFrame {i+1}: {df.shape[0]}")

    # Calculate the sum of rows in each individual dataset
    individual_row_sum = sum([df.shape[0] for df in dataframes])

    # Concatenate all DataFrames in the list
    merged_df = pd.concat(dataframes, join='inner', ignore_index=True)

    return merged_df, individual_row_sum


def clean_text(text):
    """
    De esta manera tenemos el texto sin espacios blancos extra y sobre todo con todas las palabras con capitalización correcta.
    """
    if pd.isna(text):
        return text
    text = text.strip()  # Eliminate white spaces
    text = text.lower()  # Convert to lowercase
    text = unidecode.unidecode(text)  # Remove accents
    text = re.sub('-.*-', '', text)
    text = re.sub('\s+', ' ', text)  # Eliminate extra white spaces
    text = re.sub('^\s+|\s+?$', '', text)  # Eliminate spaces at the beginning and end
    return text

# 1. Lectura de los datos

### Lectura del dataset del INEGI

In [3]:
path_dataset_inegi = '../../data/dataset_inegi.csv'
dataset_inegi = pd.read_csv(path_dataset_inegi, encoding='cp1252', dtype={'CVE_ENT': str, 'CVE_MUN': str})

### Lectura del listado de Beneficiarios 2023

In [4]:
listado_beneficiarios, sumOfRows = load_datasets("../../data/productores_beneficiarios")

Number of rows in DataFrame 1: 42854
Number of rows in DataFrame 2: 493598
Number of rows in DataFrame 3: 830761
Number of rows in DataFrame 4: 312892


In [5]:
sumOfRows

1680105

In [6]:
listado_beneficiarios

,ESTADO,MUNICIPIO,ACUSE ESTATAL,APELLIDO PATERNO,APELLIDO MATERNO,NOMBRE (S),PAQUETE
0,SINALOA,AHOME,23-PROESFE-ESTR-000051-E000-SL,ABOITES,ARMENTA,FRANCISCO,4.0
1,SINALOA,AHOME,23-PROESFE-ESTR-043733-E000-SL,ABOYTE,RUIZ,ISMAEL,7.0
2,SINALOA,AHOME,23-PROESFE-ESTR-000052-E000-SL,ABOYTES,ARMENTA,RODOLFO,4.0
3,SINALOA,AHOME,23-PROESFE-ESTR-030724-E000-SL,ACOSTA,BUELNA,EDGAR,10.0
4,SINALOA,AHOME,23-PROESFE-ESTR-035928-E000-SL,ACOSTA,BUELNA,GUADALUPE,5.0
...,...,...,...,...,...,...,...
1680100,ZACATECAS,ZACATECAS,23-PRONAFE-FERT-035894-S000-ZS,VILLANEDA,ROBLES,ADOLFO,2.0
1680101,ZACATECAS,ZACATECAS,23-PRONAFE-FERT-038259-S000-ZS,VILLEGAS,PUENTE,RITO,2.0
1680102,ZACATECAS,ZACATECAS,23-PRONAFE-FERT-040710-S000-ZS,ZAMBRANO,IBARRA,JOSE DE JESUS,2.0
1680103,ZACATECAS,ZACATECAS,23-PRONAFE-FERT-035895-S000-ZS,ZAVALA,CHAVEZ,ANTONIO,2.0


In [7]:
medellin_in_dataset_inegi = 'Medellín' in dataset_inegi['NOM_MUN'].values
medellin_in_listado_beneficiarios = 'MEDELLIN' in listado_beneficiarios['MUNICIPIO'].values

print(f"MEDELLIN in dataset_inegi: {medellin_in_dataset_inegi}")
print(f"MEDELLIN in listado_beneficiarios: {medellin_in_listado_beneficiarios}")

MEDELLIN in dataset_inegi: False
MEDELLIN in listado_beneficiarios: True


# 2. Limpieza de los datos

### 2.1 INEGI

In [8]:
# Revisamos las columnas del dataset
dataset_inegi.columns

Index(['MAPA', 'Estatus', 'CVE_ENT', 'NOM_ENT', 'NOM_ABR', 'CVE_MUN',
       'NOM_MUN', 'CVE_LOC', 'NOM_LOC', 'AMBITO', 'LATITUD', 'LONGITUD',
       'LAT_DECIMAL', 'LON_DECIMAL', 'ALTITUD', 'CVE_CARTA', 'POB_TOTAL',
       'POB_MASCULINA', 'POB_FEMENINA', 'TOTAL DE VIVIENDAS HABITADAS'],
      dtype='object')

In [9]:
# Revisamos las primeras filas del dataset
dataset_inegi.head()

,MAPA,Estatus,CVE_ENT,NOM_ENT,NOM_ABR,CVE_MUN,NOM_MUN,CVE_LOC,NOM_LOC,AMBITO,LATITUD,LONGITUD,LAT_DECIMAL,LON_DECIMAL,ALTITUD,CVE_CARTA,POB_TOTAL,POB_MASCULINA,POB_FEMENINA,TOTAL DE VIVIENDAS HABITADAS
0,10010001,NaN,01,Aguascalientes,Ags.,001,Aguascalientes,1,Aguascalientes,U,"21°52´47.362N""","102°17´45.768W""",21.879822,-102.296046,1878,F13D19,863893,419168,444725,246259
1,10010094,NaN,01,Aguascalientes,Ags.,001,Aguascalientes,94,Granja Adelita,R,"21°52´18.749N""","102°22´24.710W""",21.871874,-102.373530,1901,F13D18,5,*,*,2
2,10010096,NaN,01,Aguascalientes,Ags.,001,Aguascalientes,96,Agua Azul,R,"21°53´01.522N""","102°21´25.639W""",21.883756,-102.357122,1861,F13D18,41,24,17,12
3,10010100,NaN,01,Aguascalientes,Ags.,001,Aguascalientes,100,Rancho Alegre,R,"21°51´16.556N""","102°22´21.884W""",21.854599,-102.372745,1879,F13D18,0,0,0,0
4,10010102,NaN,01,Aguascalientes,Ags.,001,Aguascalientes,102,Los Arbolitos [Rancho],R,"21°46´48.650N""","102°21´26.261W""",21.780180,-102.357295,1861,F13D18,8,*,*,2


In [10]:
# Eliminamos las columnas que no son de interés
COLUMNS_TO_DROP = ['MAPA', 'Estatus', 'NOM_ABR', 'CVE_LOC', 'NOM_LOC', 'AMBITO', 'LATITUD', 'LONGITUD',
                   'LAT_DECIMAL', 'LON_DECIMAL', 'ALTITUD', 'CVE_CARTA', 'POB_TOTAL',
                   'POB_MASCULINA', 'POB_FEMENINA', 'TOTAL DE VIVIENDAS HABITADAS']
dataset_inegi = dataset_inegi.drop(COLUMNS_TO_DROP, axis=1)

In [11]:
# Las claves de entidad y municipio serán tratadas numéricamente en la limpieza aunque posteriormente se les asignará el tipo de cadena de texto para tener el estándar.
dataset_inegi.dtypes

CVE_ENT    object
NOM_ENT    object
CVE_MUN    object
NOM_MUN    object
dtype: object

In [12]:
# Revisamos la cantidad de filas y columnas del dataset
print("Shape of dataset_inegi: ", dataset_inegi.shape)

dataset_inegi_clean = dataset_inegi.drop_duplicates()
print("Shape of dataset_inegi_clean: ", dataset_inegi_clean.shape)

Shape of dataset_inegi:  (299568, 4)
Shape of dataset_inegi_clean:  (2476, 4)


A partir de aquí seguimos trabajando con el listado de Estados y Municipios limpio de Inegi (sin repetir) "dataset_inegi_clean".

In [13]:
# Revisamos las primeras filas del dataset con las columnas seleccionadas
dataset_inegi_clean.head()

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN
0,01,Aguascalientes,001,Aguascalientes
708,01,Aguascalientes,002,Asientos
945,01,Aguascalientes,003,Calvillo
1237,01,Aguascalientes,004,Cosío
1330,01,Aguascalientes,005,Jesús María


In [14]:
# Revisamos las últimas filas del dataset con las columnas seleccionadas
dataset_inegi_clean.tail()

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN
299150,32,Zacatecas,054,Villa Hidalgo
299211,32,Zacatecas,055,Villanueva
299363,32,Zacatecas,056,Zacatecas
299484,32,Zacatecas,057,Trancoso
299526,32,Zacatecas,058,Santa María de la Paz


In [15]:
dataset_inegi_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2476 entries, 0 to 299526
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   CVE_ENT  2476 non-null   object
 1   NOM_ENT  2476 non-null   object
 2   CVE_MUN  2476 non-null   object
 3   NOM_MUN  2476 non-null   object
dtypes: object(4)
memory usage: 96.7+ KB


In [16]:
print("Los valores únicos en cada columna son:\n", dataset_inegi_clean.nunique())

Los valores únicos en cada columna son:
 CVE_ENT      32
NOM_ENT      32
CVE_MUN     570
NOM_MUN    2332
dtype: int64


In [17]:
# Creamos una columna con la clave única por municipio

dataset_inegi_clean['CVE_MUN_Unique'] = dataset_inegi_clean['CVE_ENT'].astype(str) + '-' + dataset_inegi_clean[
    'CVE_MUN'].astype(str)

dataset_inegi_clean.head()

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_31385/2609109509.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_clean['CVE_MUN_Unique'] = dataset_inegi_clean['CVE_ENT'].astype(str) + '-' + dataset_inegi_clean[


,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,CVE_MUN_Unique
0,01,Aguascalientes,001,Aguascalientes,01-001
708,01,Aguascalientes,002,Asientos,01-002
945,01,Aguascalientes,003,Calvillo,01-003
1237,01,Aguascalientes,004,Cosío,01-004
1330,01,Aguascalientes,005,Jesús María,01-005


In [18]:
dataset_inegi_clean.tail()

,CVE_ENT,NOM_ENT,CVE_MUN,NOM_MUN,CVE_MUN_Unique
299150,32,Zacatecas,054,Villa Hidalgo,32-054
299211,32,Zacatecas,055,Villanueva,32-055
299363,32,Zacatecas,056,Zacatecas,32-056
299484,32,Zacatecas,057,Trancoso,32-057
299526,32,Zacatecas,058,Santa María de la Paz,32-058


#### 2.1.1 Estandarización de nombre de municipios

Con el fin de poder hacer un merge bajo los mismos nombres, hacemos una limpieza de los datos.

In [19]:
# Estandarizamos la limpieza de los datos
dataset_inegi_clean['NOM_ENT_Clean'] = dataset_inegi_clean['NOM_ENT'].apply(clean_text)
dataset_inegi_clean['NOM_MUN_Clean'] = dataset_inegi_clean['NOM_MUN'].apply(clean_text)

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_31385/1220250931.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_clean['NOM_ENT_Clean'] = dataset_inegi_clean['NOM_ENT'].apply(clean_text)
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_31385/1220250931.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_clean['NOM_MUN_Clean'] = dataset_inegi_clean['NOM_MUN'].apply(clean_text)


## 2.2 Beneficiarios 2023

### Creación de Estados_Beneficiarios2023
Este dataset es una versión de Beneficiarios2023 pero más ligera y sin repeticiones.

In [20]:
listado_beneficiarios.columns

Index(['ESTADO', 'MUNICIPIO', 'ACUSE ESTATAL', 'APELLIDO PATERNO',
       'APELLIDO MATERNO', 'NOMBRE (S)', 'PAQUETE'],
      dtype='object')

In [21]:
listado_beneficiarios.dtypes

ESTADO               object
MUNICIPIO            object
ACUSE ESTATAL        object
APELLIDO PATERNO     object
APELLIDO MATERNO     object
NOMBRE (S)           object
PAQUETE             float64
dtype: object

In [22]:
descriptive_stats = listado_beneficiarios.describe(include='all').transpose()

# Mostrar las estadísticas descriptivas
print(descriptive_stats)

                      count   unique                             top    freq  \
ESTADO              1680105       30                        GUERRERO  331097   
MUNICIPIO           1680105     2237                  LAS MARGARITAS   18041   
ACUSE ESTATAL       1680105  1680105  23-PROESFE-ESTR-000051-E000-SL       1   
APELLIDO PATERNO    1680104    14995                       HERNANDEZ   77418   
APELLIDO MATERNO    1653877    17604                       HERNANDEZ   76953   
NOMBRE (S)          1680105   124628                            JUAN   26798   
PAQUETE           1680097.0      NaN                             NaN     NaN   

                      mean       std  min  25%  50%  75%   max  
ESTADO                 NaN       NaN  NaN  NaN  NaN  NaN   NaN  
MUNICIPIO              NaN       NaN  NaN  NaN  NaN  NaN   NaN  
ACUSE ESTATAL          NaN       NaN  NaN  NaN  NaN  NaN   NaN  
APELLIDO PATERNO       NaN       NaN  NaN  NaN  NaN  NaN   NaN  
APELLIDO MATERNO       NaN       N

In [23]:
municipios_null = listado_beneficiarios[listado_beneficiarios['MUNICIPIO'].isnull()]

# ¿Qué ESTADOS tienen algún municipio Null?
print(municipios_null['ESTADO'].unique())

[]


In [24]:
duplicated_rows_productores = listado_beneficiarios.duplicated()
number_of_duplicated_rows_productores = duplicated_rows_productores.sum()
print(f"El número de filas duplicadas es: {number_of_duplicated_rows_productores}")
print(f"El número de filas únicas es: {listado_beneficiarios.shape[0] - number_of_duplicated_rows_productores}")

El número de filas duplicadas es: 0
El número de filas únicas es: 1680105


In [25]:
listado_beneficiarios.isna().sum()

ESTADO                  0
MUNICIPIO               0
ACUSE ESTATAL           0
APELLIDO PATERNO        1
APELLIDO MATERNO    26228
NOMBRE (S)              0
PAQUETE                 8
dtype: int64

In [26]:
listado_beneficiarios

,ESTADO,MUNICIPIO,ACUSE ESTATAL,APELLIDO PATERNO,APELLIDO MATERNO,NOMBRE (S),PAQUETE
0,SINALOA,AHOME,23-PROESFE-ESTR-000051-E000-SL,ABOITES,ARMENTA,FRANCISCO,4.0
1,SINALOA,AHOME,23-PROESFE-ESTR-043733-E000-SL,ABOYTE,RUIZ,ISMAEL,7.0
2,SINALOA,AHOME,23-PROESFE-ESTR-000052-E000-SL,ABOYTES,ARMENTA,RODOLFO,4.0
3,SINALOA,AHOME,23-PROESFE-ESTR-030724-E000-SL,ACOSTA,BUELNA,EDGAR,10.0
4,SINALOA,AHOME,23-PROESFE-ESTR-035928-E000-SL,ACOSTA,BUELNA,GUADALUPE,5.0
...,...,...,...,...,...,...,...
1680100,ZACATECAS,ZACATECAS,23-PRONAFE-FERT-035894-S000-ZS,VILLANEDA,ROBLES,ADOLFO,2.0
1680101,ZACATECAS,ZACATECAS,23-PRONAFE-FERT-038259-S000-ZS,VILLEGAS,PUENTE,RITO,2.0
1680102,ZACATECAS,ZACATECAS,23-PRONAFE-FERT-040710-S000-ZS,ZAMBRANO,IBARRA,JOSE DE JESUS,2.0
1680103,ZACATECAS,ZACATECAS,23-PRONAFE-FERT-035895-S000-ZS,ZAVALA,CHAVEZ,ANTONIO,2.0


In [27]:
# Observamos los valores únicos y la cantidad de cada columna, ordenados
COLUMNS_TO_VIEW = ['ESTADO', 'MUNICIPIO', 'PAQUETE']
for column in COLUMNS_TO_VIEW:
    unique_values = listado_beneficiarios[column].unique()
    if unique_values.dtype == 'object':
        unique_values = unique_values.astype(str)
        unique_values.sort()
    if len(unique_values) <= 1000:
        print(
            f"\n{column}\nCantidad de valores unicos {len(unique_values)}. Valores únicos en la columna {column}: {unique_values}")
    else:
        print(f"\nCantidad de valores únicos en {column}: {len(unique_values)}")
        print(f"Valores únicos en la columna {column}:")
        for value in unique_values:
            print(value)


ESTADO
Cantidad de valores unicos 30. Valores únicos en la columna ESTADO: ['AGUASCALIENTES' 'CAMPECHE' 'CHIAPAS' 'CHIHUAHUA' 'CIUDAD DE MEXICO'
 'COAHUILA DE ZARAGOZA' 'COLIMA' 'DURANGO' 'GUANAJUATO' 'GUERRERO'
 'HIDALGO' 'JALISCO' 'MEXICO' 'MICHOACAN DE OCAMPO' 'MORELOS' 'NAYARIT'
 'NUEVO LEON' 'OAXACA' 'PUEBLA' 'QUERETARO DE ARTEAGA' 'QUINTANA ROO'
 'SAN LUIS POTOSI' 'SINALOA' 'SONORA' 'TABASCO' 'TAMAULIPAS' 'TLAXCALA'
 'VERACRUZ DE IGNACIO DE LA LLAVE' 'YUCATAN' 'ZACATECAS']

Cantidad de valores únicos en MUNICIPIO: 2237
Valores únicos en la columna MUNICIPIO:
ABALA
ABASOLO
ABEJONES
ACACOYAGUA
ACAJETE
ACALA
ACAMBARO
ACAMBAY
ACANCEH
ACAPETAHUA
ACAPONETA
ACAPULCO DE JUAREZ
ACATENO
ACATEPEC
ACATIC
ACATLAN
ACATLAN DE JUAREZ
ACATLAN DE PEREZ FIGUEROA
ACATZINGO
ACAXOCHITLAN
ACAYUCAN
ACOLMAN
ACONCHI
ACTEOPAN
ACTOPAN
ACUAMANALA DE MIGUEL HIDALGO
ACUITZIO
ACULA
ACULCO
ACULTZINGO
AGUA BLANCA DE ITURBIDE
AGUA DULCE
AGUA PRIETA
AGUASCALIENTES
AGUILILLA
AHOME
AHUACATLAN
AHUACUOTZINGO
AHUALULCO

In [28]:
Estados_beneficiarios = listado_beneficiarios[['ESTADO', 'MUNICIPIO']]

In [29]:
# Estandarizamos la limpieza de los datos
Estados_beneficiarios['ESTADO_Clean'] = Estados_beneficiarios['ESTADO'].apply(clean_text)
Estados_beneficiarios['MUNICIPIO_Clean'] = Estados_beneficiarios['MUNICIPIO'].apply(clean_text)

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_31385/2688933659.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Estados_beneficiarios['ESTADO_Clean'] = Estados_beneficiarios['ESTADO'].apply(clean_text)
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_31385/2688933659.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Estados_beneficiarios['MUNICIPIO_Clean'] = Estados_beneficiarios['MUNICIPIO'].apply(clean_text)


In [30]:
# Valores únicos y la cantidad de cada columna
# Obtener estadísticas descriptivas para todas las variables

descriptive_stats = Estados_beneficiarios.describe(include='all').transpose()

# Mostrar las estadísticas descriptivas
print(descriptive_stats)

                   count unique             top    freq
ESTADO           1680105     30        GUERRERO  331097
MUNICIPIO        1680105   2237  LAS MARGARITAS   18041
ESTADO_Clean     1680105     30        guerrero  331097
MUNICIPIO_Clean  1680105   2235  las margaritas   18041


# 3. Diccionario de los datasets de INEGI Y LISTADO BENEFICIARIOS 2023

El objetivo de esta sección es crear un diccionario de códigos según INEGI para los municipios Listado_beneficiarios2023. Para ello haremos un Left join entre Estados_Beneficiarios2023 y dataset_inegi_clean.

### 3.1 Creamos las columnas clave

Crearemos las columnas clave concatenando [Estado_limpio]-[Municipio_limpio] en **ambos datasets**

In [31]:
# INEGI
dataset_inegi_clean["NOM_ENT_Clean"] = dataset_inegi_clean["NOM_ENT_Clean"].astype(str)
dataset_inegi_clean["NOM_MUN_Clean"] = dataset_inegi_clean["NOM_MUN_Clean"].astype(str)

dataset_inegi_clean["KEY_inegi"] = dataset_inegi_clean["NOM_ENT_Clean"] + "-" + dataset_inegi_clean["NOM_MUN_Clean"]

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_31385/627242104.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_clean["NOM_ENT_Clean"] = dataset_inegi_clean["NOM_ENT_Clean"].astype(str)
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_31385/627242104.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_inegi_clean["NOM_MUN_Clean"] = dataset_inegi_clean["NOM_MUN_Clean"].astype(str)
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_31385/627242104.py:5: S

In [32]:
Estados_beneficiarios["ESTADO_Clean"] = Estados_beneficiarios["ESTADO_Clean"].astype(str)
Estados_beneficiarios["MUNICIPIO_Clean"] = Estados_beneficiarios["MUNICIPIO_Clean"].astype(str)

Estados_beneficiarios["KEY_benef23"] = Estados_beneficiarios["ESTADO_Clean"] + "-" + Estados_beneficiarios["MUNICIPIO_Clean"]

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_31385/2440238580.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Estados_beneficiarios["ESTADO_Clean"] = Estados_beneficiarios["ESTADO_Clean"].astype(str)
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_31385/2440238580.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Estados_beneficiarios["MUNICIPIO_Clean"] = Estados_beneficiarios["MUNICIPIO_Clean"].astype(str)
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_31385/24402

In [33]:
df = Estados_beneficiarios['KEY_benef23'].unique()

In [34]:
df.shape

(2356,)

### 3.1 Left join

Creamos el diccionario.

In [35]:
Estados_beneficiarios = Estados_beneficiarios.drop(['ESTADO', 'MUNICIPIO'], axis=1)
Estados_beneficiarios = Estados_beneficiarios.drop_duplicates()
Estados_beneficiarios.shape

(2356, 3)

In [36]:
Estados_beneficiarios

,ESTADO_Clean,MUNICIPIO_Clean,KEY_benef23
0,sinaloa,ahome,sinaloa-ahome
2574,sinaloa,angostura,sinaloa-angostura
3789,sinaloa,badiraguato,sinaloa-badiraguato
6623,sinaloa,choix,sinaloa-choix
9113,sinaloa,concordia,sinaloa-concordia
...,...,...,...
1672843,zacatecas,tlaltenango de sanchez roman,zacatecas-tlaltenango de sanchez roman
1673139,zacatecas,trancoso,zacatecas-trancoso
1673741,zacatecas,trinidad garcia de la cadena,zacatecas-trinidad garcia de la cadena
1674556,zacatecas,vetagrande,zacatecas-vetagrande


In [37]:
# Crear una función para encontrar la mejor coincidencia difusa con límites entre 90 y 100 de coincidencia
def fuzzy_merge_beneficiarios2023(df_inegi, df_prod, key1, key2, threshold=90, limit=1):
    """
    df_inegi: DataFrame de la izquierda (el DataFrame principal)
    df_prod: DataFrame de la derecha (el DataFrame con el que se quiere hacer el join)
    key1: Columna de la clave en df_inegi
    key2: Columna de la clave en df_prod
    threshold: Umbral de coincidencia difusa
    limit: Número de coincidencias a encontrar
    """
    s = df_prod[key2].tolist()

    # Encontrar las mejores coincidencias para cada clave en df_inegi
    matches = df_inegi[key1].apply(lambda x: process.extractOne(x, s, score_cutoff=threshold))

    # Crear una columna con las mejores coincidencias
    df_inegi['best_match'] = [match[0] if match else None for match in matches]
    df_inegi['match_score'] = [match[1] if match else None for match in matches]

    # Hacer el merge con las mejores coincidencias
    df_merged = pd.merge(df_inegi, df_prod, left_on='best_match', right_on=key2, how='inner',
                         suffixes=('_inegi', '_prod'))
    
    return df_merged


In [38]:
# Aplicar la función de coincidencia difusa
diccionario = fuzzy_merge_beneficiarios2023(dataset_inegi_clean, Estados_beneficiarios, 'KEY_inegi', 'KEY_benef23')
diccionario.drop_duplicates(subset=['KEY_inegi'], inplace=True)

# Mostrar el resultado
diccionario.columns

/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_31385/1688351057.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inegi['best_match'] = [match[0] if match else None for match in matches]
/var/folders/ng/n9ng0wy92t90_49j662nh02c0000gn/T/ipykernel_31385/1688351057.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_inegi['match_score'] = [match[1] if match else None for match in matches]


Index(['CVE_ENT', 'NOM_ENT', 'CVE_MUN', 'NOM_MUN', 'CVE_MUN_Unique',
       'NOM_ENT_Clean', 'NOM_MUN_Clean', 'KEY_inegi', 'best_match',
       'match_score', 'ESTADO_Clean', 'MUNICIPIO_Clean', 'KEY_benef23'],
      dtype='object')

In [39]:
nan_rows = diccionario.isna()

In [40]:
diccionario.to_csv('../../data/diccionario.csv', index=False)

## 3.2 Listado beneficiarios2023

Esta sección se encarga de completar el listado original de Beneficiarios Autorizados con los nombre corregido de INEGI usando el diccionario.

In [41]:
# Crear una variable KEY en listado de productores y el diccionario para hacer el join
listado_beneficiarios['ESTADO_Clean'] = listado_beneficiarios['ESTADO'].apply(clean_text)
listado_beneficiarios['MUNICIPIO_Clean'] = listado_beneficiarios['MUNICIPIO'].apply(clean_text)
listado_beneficiarios['Estado-mun-KEY'] = listado_beneficiarios['ESTADO_Clean'].astype(str) + '-' + listado_beneficiarios[
    'MUNICIPIO_Clean'].astype(str)

In [42]:
nan_rows = listado_beneficiarios.isna()

In [43]:
nan_rows.sum()

ESTADO                  0
MUNICIPIO               0
ACUSE ESTATAL           0
APELLIDO PATERNO        1
APELLIDO MATERNO    26228
NOMBRE (S)              0
PAQUETE                 8
ESTADO_Clean            0
MUNICIPIO_Clean         0
Estado-mun-KEY          0
dtype: int64

In [44]:
listado_beneficiarios.shape

(1680105, 10)

In [45]:
diccionario.shape

(2354, 13)

In [46]:
# Lectura del diccionario manipulado
diccionario_manipulado = pd.read_csv('../../data/Diccionario_manual.csv')

In [47]:
# Hacer el join
listado_beneficiarios_complete = pd.merge(listado_beneficiarios, diccionario, left_on="Estado-mun-KEY",
                                        right_on="KEY_benef23", how='left', suffixes=('_prod', '_inegi'))

In [48]:
listado_beneficiarios_complete[['CVE_ENT', 'CVE_MUN']] = listado_beneficiarios_complete['CVE_MUN_Unique'].str.split('-',
                                                                                                                expand=True)


In [49]:
listado_beneficiarios_complete.columns

Index(['ESTADO', 'MUNICIPIO', 'ACUSE ESTATAL', 'APELLIDO PATERNO',
       'APELLIDO MATERNO', 'NOMBRE (S)', 'PAQUETE', 'ESTADO_Clean_prod',
       'MUNICIPIO_Clean_prod', 'Estado-mun-KEY', 'CVE_ENT', 'NOM_ENT',
       'CVE_MUN', 'NOM_MUN', 'CVE_MUN_Unique', 'NOM_ENT_Clean',
       'NOM_MUN_Clean', 'KEY_inegi', 'best_match', 'match_score',
       'ESTADO_Clean_inegi', 'MUNICIPIO_Clean_inegi', 'KEY_benef23'],
      dtype='object')

In [50]:
# Seleccionamos las columnas que nos interesan
listado_beneficiarios_complete = listado_beneficiarios_complete[
    ['ESTADO', 'MUNICIPIO', 'ACUSE ESTATAL', 'APELLIDO PATERNO', 'APELLIDO MATERNO',
     'NOMBRE (S)', 'PAQUETE', 'KEY_benef23', 'NOM_ENT', 'NOM_MUN', 'CVE_ENT', 'CVE_MUN']]

In [51]:
# Revisamos el dataset
print(listado_beneficiarios_complete.shape)
print(listado_beneficiarios_complete.columns)
print(listado_beneficiarios_complete.head())

(1700942, 12)
Index(['ESTADO', 'MUNICIPIO', 'ACUSE ESTATAL', 'APELLIDO PATERNO',
       'APELLIDO MATERNO', 'NOMBRE (S)', 'PAQUETE', 'KEY_inegi', 'NOM_ENT',
       'NOM_MUN', 'CVE_ENT', 'CVE_MUN'],
      dtype='object')
    ESTADO MUNICIPIO                   ACUSE ESTATAL APELLIDO PATERNO  \
0  SINALOA     AHOME  23-PROESFE-ESTR-000051-E000-SL          ABOITES   
1  SINALOA     AHOME  23-PROESFE-ESTR-043733-E000-SL           ABOYTE   
2  SINALOA     AHOME  23-PROESFE-ESTR-000052-E000-SL          ABOYTES   
3  SINALOA     AHOME  23-PROESFE-ESTR-030724-E000-SL           ACOSTA   
4  SINALOA     AHOME  23-PROESFE-ESTR-035928-E000-SL           ACOSTA   

  APELLIDO MATERNO NOMBRE (S)  PAQUETE      KEY_inegi  NOM_ENT NOM_MUN  \
0          ARMENTA  FRANCISCO      4.0  sinaloa-ahome  Sinaloa   Ahome   
1             RUIZ     ISMAEL      7.0  sinaloa-ahome  Sinaloa   Ahome   
2          ARMENTA    RODOLFO      4.0  sinaloa-ahome  Sinaloa   Ahome   
3           BUELNA      EDGAR     10.0  sinal

In [52]:
nan_counts = listado_beneficiarios_complete.isna().sum()
print(nan_counts)

ESTADO                  0
MUNICIPIO               0
ACUSE ESTATAL           0
APELLIDO PATERNO        1
APELLIDO MATERNO    26521
NOMBRE (S)              0
PAQUETE                 8
KEY_inegi           21605
NOM_ENT             21605
NOM_MUN             21605
CVE_ENT             21605
CVE_MUN             21605
dtype: int64


In [53]:
nan_rows = listado_beneficiarios_complete[listado_beneficiarios_complete['NOM_ENT'].isna()]


In [54]:
nan_rows.tail(50)

,ESTADO,MUNICIPIO,ACUSE ESTATAL,APELLIDO PATERNO,APELLIDO MATERNO,NOMBRE (S),PAQUETE,KEY_inegi,NOM_ENT,NOM_MUN,CVE_ENT,CVE_MUN
1584442,SAN LUIS POTOSI,AHUALULCO,23-PRONAFE-FERT-000342-S000-SP,VAZQUEZ,OROZCO,MARCO ANTONIO,2.0,NaN,NaN,NaN,NaN,NaN
1584443,SAN LUIS POTOSI,AHUALULCO,23-PRONAFE-FERT-000343-S000-SP,VAZQUEZ,PALOMO,FELIPE,1.0,NaN,NaN,NaN,NaN,NaN
1584444,SAN LUIS POTOSI,AHUALULCO,23-PRONAFE-FERT-000344-S000-SP,VAZQUEZ,QUIROZ,MA. CRUZ,1.0,NaN,NaN,NaN,NaN,NaN
1584445,SAN LUIS POTOSI,AHUALULCO,23-PRONAFE-FERT-000402-S000-SP,VAZQUEZ,RAMIREZ,ANGELA,2.0,NaN,NaN,NaN,NaN,NaN
1584446,SAN LUIS POTOSI,AHUALULCO,23-PRONAFE-FERT-000345-S000-SP,VAZQUEZ,RAMIREZ,BRENDA JUDITH,2.0,NaN,NaN,NaN,NaN,NaN
1584447,SAN LUIS POTOSI,AHUALULCO,23-PRONAFE-FERT-000346-S000-SP,VAZQUEZ,RAMIREZ,EMIGDIO,2.0,NaN,NaN,NaN,NaN,NaN
1584448,SAN LUIS POTOSI,AHUALULCO,23-PRONAFE-FERT-000347-S000-SP,VAZQUEZ,RAMIREZ,JOSE ALFREDO,2.0,NaN,NaN,NaN,NaN,NaN
1584449,SAN LUIS POTOSI,AHUALULCO,23-PRONAFE-FERT-000754-S000-SP,VAZQUEZ,SANDATE,AGUSTIN,2.0,NaN,NaN,NaN,NaN,NaN
1584450,SAN LUIS POTOSI,AHUALULCO,23-PRONAFE-FERT-000755-S000-SP,VAZQUEZ,SANDATE,TERESA,2.0,NaN,NaN,NaN,NaN,NaN
1584451,SAN LUIS POTOSI,AHUALULCO,23-PRONAFE-FERT-000350-S000-SP,VAZQUEZ,SOLIS,PAULINA,1.0,NaN,NaN,NaN,NaN,NaN


In [55]:
listado_beneficiarios_complete = listado_beneficiarios_complete.dropna(subset=['PAQUETE'])

In [56]:
listado_beneficiarios_complete.shape

(1700934, 12)

In [57]:
listado_beneficiarios_complete = listado_beneficiarios_complete.astype({
    'ESTADO': 'str',
    'MUNICIPIO': 'str',
    'ACUSE ESTATAL': 'str',
    'APELLIDO PATERNO': 'str',
    'APELLIDO MATERNO': 'str',
    'NOMBRE (S)': 'str',
    'PAQUETE': 'int',
    'NOM_MUN': 'str',
    'NOM_ENT': 'str',
    'CVE_MUN': 'str',
    'CVE_ENT': 'str',
    'KEY_benef23': 'str'

    })

listado_beneficiarios_complete = listado_beneficiarios_complete.rename(columns={
'ESTADO': 'estado1',
'MUNICIPIO': 'municipio1',
'ACUSE ESTATAL': 'acuse',
'APELLIDO PATERNO': 'apellido_paterno',
'APELLIDO MATERNO': 'apellido_materno',
'NOMBRE (S)': 'nombre_propio',
'PAQUETE': 'paquete',
'NOM_MUN': 'municipio',
'NOM_ENT': 'entidad',
'CVE_MUN': 'cve_mun',
'CVE_ENT': 'cve_ent',
'KEY_benef23': 'key_benef23'
})

In [58]:
listado_beneficiarios_complete = listado_beneficiarios_complete.drop(columns=['estado1', 'municipio1'])

listado_beneficiarios_complete['id'] = listado_beneficiarios_complete.index

# Assuming df is your DataFrame
ordered_columns = ['id', 'cve_ent', 'entidad', 'cve_mun', 'municipio', 'acuse', 'apellido_paterno', 'apellido_materno', 'nombre_propio', 'paquete', 'key_benef23']
listado_beneficiarios_complete = listado_beneficiarios_complete.reindex(columns=ordered_columns)

listado_beneficiarios_complete['cve_ent'] = listado_beneficiarios_complete['cve_ent'].str.zfill(2)
listado_beneficiarios_complete['cve_mun'] = listado_beneficiarios_complete['cve_mun'].str.zfill(3) 

In [59]:
listado_beneficiarios_complete.to_csv('../../data/listado_beneficiarios_complete.csv', index=False)